[Lecture 2: Recurrent Neural Networks and Transformers](https://www.youtube.com/watch?v=QvkQ1B3FBqA&list=PLtBw6njQRU-rwp5__7C0oIVt26ZgjG9NI)

goal: to make use of the fact that some data is sequential in making predictions.

it could be that our input is temporal or generally sequential, or that our output is.

the output at any given timestep depends on the input at that timestep.  
with sequential data, we also need to find a way to honor the fact that the output at t2 depends on the input at t1 and t0, not just at t2.

we do this by adding the concept of internal memory / state = h(t).  
our output depends not only on the input at that time, but on the memory from the prior time.  
$\hat{y}(t) = f(x_t, h_{t-1})$  
output depends on input and on past memory.  
cool!  

$h_t = f_W(x_t, h_{t-1})$

cell state is a function of weights, our current input, and our prior cell state.  
the weights are fixed for some reason...I suppose we only do gradient descent updating after a whole sequence of data is processed...we can't do training in the middle of some set of sequential data.  

^ is exactly right. we iterate over each piece (datum?) of the sequence of data before making a prediction. the sequence of data is sort of a single datum in this model. we use the recurrence relation to keep track of what's gone on since the beginning of the input. 

we have one weight matrix that we multiply by the past state, and another weight matrix that we multiply by the current input. we add the values of these matrix multiplication operations, take some sigmoid thing of them, and that gives our current state.

the same weight matrices are used at each time step.
we calculate the loss after each time step and sum them (but do NOT update the weight matrix on the basis of that loss calculation).

to model sequences, we need to:  
1. be able to deal with variable-length sequences.  
2. track long-term dependencies. (doesn't the model we get just track the immediate-past dependencies? it keeps track of the far past insofar as the immediate past is informed by the far past, but it does not keep trak of the far past itself. right? it would be easy to fix this by making our memory larger and making t_n a function of x_n, h_n-1, and h_n-1 [and so on if we want to keep track of longer-term dependencies].)  
3. maintain information about order. (this makes a lot of sense. in FFNNs it doesn't matter what order the data is fed in, which makes little sense in many contexts.)
4. share parameters across the sequence.  

RNNs meet all these criteria!

embedding: transform indices into a vector of fixed size.  
1. vocabulary embedded by mapping a work to an index. one way to do this: make the length of the input vector the size of the vocabulary. for any input, every value in the vector is 0, except for one, which is a 1. "one-hot embedding."
2. take in a bunch of words and give them indices in some lower-dimensional space using unsupervised learning. this encoding gives similar words have similar values. "learned embedding."

FFNNs can't handle variable-length inputs, while RNNs can.

backpropagation through time how RNNs are trained.

we multiply the gradient backward through our model to update our weights. 2 problems:  
1. exploding gradients. the product gets too big. solution: we trim the gradient.  
2. vanishing gradients. the weightts are too close to 0. solutions: choosing our activation function and initializing our weights intelligently; designing our network to mitigate this problem altogether. (how is all of this done?)

if our gradients vanish because we are multiplying many small numbers together, the model is biased to capturing short-term dependencies.

1. activation functions.  ReLU prevents f' from shrinking when x > 0. this keeps gradients from vanishing near 0.

2. initialize the weights to the identity matrix and the biases to 0 to keep the weigths from shrinking to 0.

3. use a more complicated model that can better track the long-term dependencies in the data. use gates to add or remove information within each recurrent unit. LSTMs are a common model used for this. they have the same chain structure but the computation within each cell is more complicated.

LSTMs: key concepts.  
1. maintain a cell state.  
2. forget gate gets rid of irrelevant information. cell stores relevant information and selectively updates its state on the basis of said info. the output gate returns a filtered version of the cell state.  
backpropagation through time for some reason becomes much more stable.

this architecture also works on classification tasks on sequential data.

limitations of RNNs:  
1. encoding bottleneck. information lost in encoding operation.  
2. slow; no parallelization.  
3. memory still not very big. limited to 10s of data at most, normally.  

instead, we would like:
1. continuous stream of data.  
2. parallelization.
3. super-long memory.

we could have all this if we eliminated the need for the recurrence.

attention: identify and attend to the most important part of an input.

attention cornerstone of transformer architecture.

don't just scan inputs linearly. identify which parts to attend to, then extract features from those parts with high attention.  
this "identify" step is very similar to search.  
compare some query to the each of the keys in a list. 
if the key is similar to our query, look more closely at the associated value.

we need an embedding that captures some notion of position. THIS ALLOWS US TO ACCOUNT FOR SEQUENTIAL DATA WITHOUT DOING COMPUTATIONALLY EXPENSIVE RECURRENCE CALCULATIONS. right?

positional embedding x linear layer = output = query.  
positional embedding x linear layer = output = key.  
positional embedding x linear layer = output = value.  
in other words, we have some positional embedding, which means that it's an embedding that accounts for the order of the inputs. we put the positional embedding of the query, of the key, and of the value through the linear layer. we calculate the pairwise similarity between each query and key. this tells us how cloes the key is to what we're searching for.  
we can calculate similarity with the dot product.

we get the attention weighting by calculating the self-similarity of the query matrix.  
multiply this by the value to get an output. this is what we feed into our NN.

transformers use attention.

reflections

RNNs use recurrence to keep track of sequential data using a cell state at t, which, along with the input at t+1, determines the output at t+1.  
the backpropagation algorithm is messy and I don't really understand it and we do backpropagation backward through time but also instantaneously because not all of the output is time-dependent. right? I need to study this more.  
I half understand attention. this is a way of avoiding recurrence, which is annoying for several reasons. attention helps us to focus on the parts of the input that are most similar by calculating the self-attention matrix for an query and multiplying that by each input to extract only those parts of the input that are most relevant.  

to-do: study up on LSTMs, back-propagation through time, and attention.  
also to-do: build these models!